In [3]:
inputExample = '389125467'
inputReal = '871369452'

from collections import deque
import cProfile, pstats, io
from pstats import SortKey
import datetime

In [4]:
def part1(input):
    cups = [int(x) for x in list(input)]

    for move in range(100):
        # print('-- move', move + 1, '--')
        cups = doRoundList(cups, 9)

    while cups[0] != 1:
        cups.append(cups.pop(0))

    cups.pop(0)
    print("".join(map(str,cups)))

In [5]:
def doRoundList(cups, maxNum):
    #print('cups:', *cups)
    currentCup = cups[0]
    #print('current:', currentCup)

    pickedUp = cups[1:4]
    #print('pick up:', *pickedUp)

    newCups = cups
    cups.pop(1)
    cups.pop(1)
    cups.pop(1)

    ## find destination
    destination = currentCup - 1
    if destination == 0:
        destination = maxNum

    while True:
        if destination in newCups:
            found = True
            break
        destination -= 1
        if destination == 0:
            destination = maxNum

    #print('destination:', destination)
    destIdx = newCups.index(destination)

    pickedUp.reverse()
    for x in pickedUp:
        newCups.insert(destIdx + 1, x)

    newCups.append(newCups.pop(0))
    #print("result:", *newCups)

    return newCups

In [6]:
part1(inputExample)

67384529


In [7]:
part1(inputReal)

28793654


## Part 2. Run it ten million times, with a million cups

OK, so we need to be a bit smarter here.

Using a list, the above code is expected to run for about 16 days. Using a deque instead, it ran for almost exactly four days (and produced the wrong answer).

For this to work, I need a low O(...) score for moving items around the list, and a low O(...) score for random access. For the former, a linked list sounds great (`O(1)`) at the cost of random access (`O(n)`). For the latter, moving around isn't great (`O(n)`) but random access is fine (`O(1)`).

I'm going to cheat a bit, and use a combination of both.

I'm going to use a list, with the **indexes** referring to the cup label minus one (such that cup 5 has index 4), and the list value refers to the **index** of the next cup in the circle. This way, I should get `O(1)` for movement, and `O(1)` for access, at the risk of off-by-one errors. It is my hope that by writing everything with indexes rather than labels, I can isolate the off-by-one errors to the parsing and the output display parts of the code.

In [21]:
def part1(input):
    cups = run_game(input, 9, 100)

    print("-- final --")
    print_cups(cups, cups[0])

def part2(input):
    cups = run_game(input, 1_000_000, 10_000_000)

    print("-- final --")
    print(cups[0] +1, cups[cups[0]] + 1)
    print((cups[0]+1) * (cups[cups[0]]+1))

In [9]:
def build_cups(input, total_cups):
    input_cups = [int(x) - 1 for x in list(input)]
    len_input = len(input_cups)
    #print("input", input_cups, len_input)
    # build initial range of cups such that each cup holds the next sequential index
    cups = list(range(1, total_cups + 1))
    # put the cups in the correct places with the correct next jumps
    first_cup = input_cups.pop(0)
    current_cup = first_cup
    while len(input_cups) > 0:
        next_cup = input_cups.pop(0)
        cups[current_cup] = next_cup
        current_cup = next_cup
    # connect the end of the input and extra cups correctly
    if len_input == total_cups:
        cups[current_cup] = first_cup
    else:
        cups[current_cup] = len_input
        cups[-1] = first_cup
    #print("data   ", cups)
    #print("indexes", list(range(total_cups)))
    #print("rebuild", end=" ")
    #print_cups(cups, 2)
    return cups, first_cup

In [10]:
def print_cups(cups, current):
    cup_list = []
    last = current
    for i in range(15):
        # print(last)
        cup_list.append(last + 1)
        last = cups[last]
        if last == current:
            break
    print("CUPS", cup_list)

In [11]:
def run_game(input, total_cups, total_moves):
    (cups, first_cup) = build_cups(input, total_cups)

    current_cup = first_cup
    for move in range(total_moves):
        #print('-- move', move + 1, '--')

        #print_cups(cups, current_cup)

        # the picked up cup(-chain)
        picked_up = cups[current_cup]
        all_picked_up = [picked_up, cups[picked_up], cups[cups[picked_up]]]

        # work out where the current cup should now link to
        next_after_pickup = cups[cups[cups[picked_up]]]

        cups[cups[cups[picked_up]]] = picked_up # make a loop
        #print("picked up ", end='')
        #print_cups(cups, picked_up)

        #print("next", next_after_pickup)
        cups[current_cup] = next_after_pickup

        destination = current_cup - 1
        if destination == -1:
            destination = total_cups - 1
        while True:
            if destination in all_picked_up:
                destination -= 1
            else:
                break
            if destination == -1:
                destination = total_cups - 1

        #print("destination", destination)

        new_target = cups[destination]
        cups[destination] = picked_up
        cups[all_picked_up[-1]] = new_target

        #print("Resulting", end=' ')
        #print_cups(cups, current_cup)

        current_cup = cups[current_cup]

    return cups


In [12]:
part1(inputExample)

-- final --
CUPS [6, 7, 3, 8, 4, 5, 2, 9, 1]


In [13]:
part1(inputReal)

-- final --
CUPS [2, 8, 7, 9, 3, 6, 5, 4, 1]


In [22]:
part2(inputExample)

-- final --
934001 159792
149245887792


In [23]:
part2(inputReal)

-- final --
385778 931123
359206768694
